In [66]:
import pandas as pd

In [132]:
path_to_file = "tscvetaeva.txt"
with open(path_to_file) as f:
    text_song = f.readlines()
#text_song

df = pd.DataFrame(text_song, columns = ["text"])
df = df[df["text"] != "\n"]
df = df.reset_index(drop = True)

l1 = []
df2 = pd.DataFrame(columns = ["text"])
for ind in df.index:
    n1 = ind + 1
    n = n1 % 4
    if n == 1: 
        l1 = [df['text'][ind]]
    else:
        l1.append(df['text'][ind])
    if n == 0: 
        text = "".join(l1)
        new_row = pd.DataFrame([text], columns=['text'])
        df2 = pd.concat([df2, new_row], ignore_index=True)
        #text = ""

df2["text"] = df2.apply(lambda row: row["text"][:-1] if row["text"][-1] else row["text"], axis = 1)
df = df2

#df2.values


In [139]:
df = pd.read_csv("паспорт хорошего русского - песня скомпилированная рифма AABB.csv",)

In [125]:
df.iloc[0].values

array(['Война против Украины – грех страшной силы,\nВыводить войска, искуплять вину  теперь такие наши дела.\nбез вольной Украины мы сами на краю могилы\nМир и справедливость пока не сгорели мы дотла.'],
      dtype=object)

In [137]:
df.iloc[0].values

array([list(['Уж сколько их упало в эту бездну', 'Разверстую вдали', 'Настанет день, когда и я исчезну', 'С поверхности земли'])],
      dtype=object)

In [140]:
def get_item(text):
    arr = text.split("\n")
    return list(map(lambda x: x if x[-1].isalpha() else x[:-1], arr))
    
#get_item(df.iloc[0].values[0])
df["text"] = df.apply(lambda raw:get_item(raw.iloc[0]), axis=1)

In [141]:
def get_rhyme_score(text1, text2):

    #This function counts only the direct equals of symbols from the end of phrases 
    #and isn`t aware of fonetical equals
    
    n = 1
    while (n < len(text1) or n < len(text2)):
        c1 = text1[-n]
        c2 = text2[-n] 
        #print(c1, c2)
        if c1 == c2:
            n = n+1
        else:
            break
    return n - 1

In [142]:
"A" , get_rhyme_score(df[0][0], df[0][1]), get_rhyme_score(df[0][0], df[0][2]), get_rhyme_score(df[0][0], df[0][3])

KeyError: 0

In [143]:
"B" , get_rhyme_score(df[0][1], df[0][2]), get_rhyme_score(df[0][1], df[0][3])

KeyError: 0

In [144]:
def get_rhyme_score_of_quatrain(l4_quatrain):
    l1 = l4_quatrain
    letters = "ABCD"
    n = len(letters)
    scores = [None] * n 
    for i in range(0,n):
        current_letter = letters[i]
        if scores[i] is None: 
#            scores[i] = {current_letter:100}
            scores[i] = {current_letter}
        for j in range(i+1,n):
            score = get_rhyme_score(l1[i], l1[j])
            if (score > 0):
                scores[j] = {current_letter:score}
    return scores

In [145]:
l1 = df.iloc[6]
l1 , get_rhyme_score_of_quatrain(l1)

/tmp/ipykernel_20036/2100053303.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  score = get_rhyme_score(l1[i], l1[j])


IndexError: index 1 is out of bounds for axis 0 with size 1

In [16]:
df["scores"] = df.apply(lambda raw: get_rhyme_score_of_quatrain(raw.iloc[0]), axis = 1)

In [17]:
df

,text,scores
0,"[Война против Украины – грех страшной силы, Вы...","[{A}, {B}, {'A': 3}, {'B': 2}]"
1,[Путинский режим вызывает ненавись в наших гла...,"[{A}, {B}, {'A': 1}, {'C': 2}]"
2,"[Власть империи внутри и за пределами земель, ...","[{A}, {'A': 1}, {C}, {'C': 2}]"
3,[Политузников и пленных надо освободить скорей...,"[{A}, {'A': 2}, {C}, {'C': 1}]"
4,"[Солидарность голосов наших решительна, Нет и...","[{A}, {'A': 2}, {C}, {'C': 2}]"
5,"[Наша Декларация ценность воли разделяя,, Уваж...","[{A}, {B}, {'A': 1}, {D}]"
6,"[Вместе стоим в рядах антивоенных, Публичному ...","[{A}, {B}, {C}, {'C': 2}]"


### Let`s control the different cases:

In [21]:
df.iloc[0].values
#here is Ok, ABAB

array([list(['Война против Украины – грех страшной силы', 'Выводить войска, искуплять вину  теперь такие наши дела', 'без вольной Украины мы сами на краю могилы', 'Мир и справедливость пока не сгорели мы дотла']),
       list([{'A'}, {'B'}, {'A': 3}, {'B': 2}])], dtype=object)

In [24]:
df.iloc[1].values
#here is ABAC, should be AABB
#because phonetically ах ~ аш

array([list(['Путинский режим вызывает ненавись в наших глазах', 'Ликвидация режима это долго, но верный выбор наш', 'Мы мечтаем о России, где права есть у всех', 'Народ без Путина, в этом первый наш успех']),
       list([{'A'}, {'B'}, {'A': 1}, {'C': 2}])], dtype=object)

In [25]:
df.iloc[6].values
#here is ABCC, should be AABB 
#because phonetically ных ~ ну

array([list(['Вместе стоим в рядах антивоенных', 'Публичному согласию мы знаем цену', 'Солидарность и единство нам важны как никогда', 'Из пепла восстанут наши города']),
       list([{'A'}, {'B'}, {'C'}, {'C': 2}])], dtype=object)

### Conclusion: the get_rhyme_score(text1, text2) should be extended for fonetical equal phrases
## theory here https://stihi.ru/2011/10/19/7548#:~:text=%D0%9D%D0%B5%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F%20%D1%80%D0%B8%D1%84%D0%BC%D0%B0%20%D1%85%D0%B0%D1%80%D0%B0%D0%BA%D1%82%D0%B5%D1%80%D0%B8%D0%B7%D1%83%D0%B5%D1%82%D1%81%D1%8F%20%D1%80%D0%B0%D0%B7%D0%BB%D0%B8%D1%87%D0%B8%D1%8F%D0%BC%D0%B8%20%D0%B2,%D1%81%D0%BB%D0%BE%D0%B3%D0%B0%D1%85%2C%20%D1%80%D0%B0%D1%81%D0%BF%D0%BE%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%BD%D1%8B%D1%85%20%D0%B2%20%D0%BE%D0%BA%D0%BE%D0%BD%D1%87%D0%B0%D0%BD%D0%B8%D1%8F%D1%85%20%D1%81%D1%82%D1%80%D0%BE%D0%BA.


In [5]:
def is_Consonance():

    #https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BD%D1%81%D0%BE%D0%BD%D0%B0%D0%BD%D1%81_(%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8F)
    return False

def is_Assonance():

    #https://en.wikipedia.org/wiki/Assonance
    return False
    
